In [1]:
# Import Dependencies
import requests
import json
import pandas as pd
import time
import numpy as np
from config import yelp_key

In [2]:
# API Base URL
base_url = "https://api.yelp.com/v3/businesses/search?"

In [3]:
# API Headers
headers = {
    "accept" : "application/json",
    "Authorization" : "Bearer " + yelp_key 
}

In [4]:
# Create List of Categories (manually located)
categories = ['beaches','hanggliding','horsebackriding',
             'hiking','hot_air_balloons',
              'paddleboarding','parasailing',
              'sailing','snorkeling','ziplining', 
              'spas','hotsprings','massage','bedbreakfast','hotels','resorts',
              'skiresorts','tours', 'transportation','restaurants','nightlife'
             ]

len(categories)

21

In [5]:
# Import Lat/Long Database
csv_path = "../Resources/CitiesWGeolocation.csv"
locations_df = pd.read_csv(csv_path)
locations_df.head()

,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,0,Lexington,Kentucky,United States,38.046407,-84.497039
1,1,San Diego,California,United States,32.717420,-117.162773
2,2,Cook Islands​,NaN,Cook Islands​,-19.996972,-157.785871
3,3,Park City,Utah,United States,40.646092,-111.497996
4,4,Newcastle Upon Tyne,England,United Kingdom,54.973847,-1.613157


In [6]:
# Gather Lists of Latitudes and Longitudes
latitudes_list = []
longitudes_list = []
for i in range(locations_df.shape[0]):
    latitudes_list.append(locations_df.loc[i].at["Latitude"])
    longitudes_list.append(locations_df.loc[i].at["Longitude"])
print(latitudes_list)
print(longitudes_list)

[38.0464066, 32.7174202, -19.99697155, 40.646092100000004, 54.97384739999999, -27.468968199999996, -28.0402165, 34.244058700000004, -36.718804999999996, 49.5041747, 42.64873625, 37.2395367, -32.728465, 17.223472100000002, 25.265347100000003, 50.67108245, 21.721746, 36.508976000000004, 32.0564572, -16.484598300000002, -33.928992, -19.1421421, -33.934444, -28.648333299999997, -24.7761086, -45.0321923, 33.77217945, 54.70235450000001, -6.166490799999999, -20.2759451, 45.437190799999996, 19.70318225, -4.6574977, -28.002373100000003, 44.4654236, 31.6258257, nan, 37.9374939, 49.8879177, 43.10656029999999, 30.2711224, nan, 15.2214956, 43.4832523, -33.953177600000004, 26.9154576, 46.603354, 13.1500331, 20.169626800000003, 51.53882410000001, 60.39430550000001, 24.578720999999998, nan, -34.61341495, 34.862942600000004, nan, 18.1850507, nan, 12.51756625, -35.14181285, -34.4175, 46.7985624, -33.8611665, -16.9206657, 44.958452799999996, -26.6544338, -34.427808299999995, 51.08668970000001, 44.279621,

In [7]:
# Gather List of URLs
url_list = []
lat_lon_list = []
cat_list = []
for i in range(len(latitudes_list)):
        for x in range(len(categories)):
            latitude = latitudes_list[i]
            longitude = longitudes_list[i]
            category = categories[x]
            # Limit radius to appx 15 miles from the coordinates
            complete_url = base_url + "latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&radius=24140&categories=" + category + "&sort_by=best_match"
            url_list.append(complete_url)
            lat_lon_list.append([latitude,longitude])
            cat_list.append(category)
# Confirm list of API calls is less than 5,000 daily limit
print(len(url_list))
url_list

4977


['https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=beaches&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hanggliding&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=horsebackriding&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hiking&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=hot_air_baloons&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=paddleboarding&sort_by=best_match',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=24140&categories=parasailing&sort_by=best_match

In [8]:
# Create API Calls (separate the blank list to a new cell so we can do numerous calls if needed)
data_list=[]

In [13]:
record_count = 1
set_count = 1
url_errors = []

for i in range(756,len(url_list)):
    if (i%50 == 0 and i >=50):
        set_count += 1
        record_count = 0
        
    print("processing Record %s of Set %s | %s" %(record_count, set_count,url_list[i]))    
    
    record_count += 1
    try:
        response = requests.get(url_list[i],headers=headers)
        data = response.json()

        # Calculate Average Rating and Review Count
        rating_sum = 0
        review_sum = 0
        # setting the range with an initial number so it can be changed if an error pops up midway through
        for x in range(len(data["businesses"])):
            rating_sum += data["businesses"][x]["rating"]
            review_sum += data["businesses"][x]["review_count"]
        rating_avg = rating_sum / len(data["businesses"])
        review_avg = review_sum / len(data["businesses"])
    
        # Add the desired data to a dictionary
        data_dict = {
            "latitude" : lat_lon_list[i][0] ,
            "longitude": lat_lon_list[i][1],
            "category" : cat_list[i],
            "results_total" : len(data["businesses"]),
            "avg_rating" : rating_avg,
            "avg_review_count" : review_avg
        }
        # Append the dictionary to a list
        data_list.append(data_dict)
        time.sleep(1)
    except:
        print("********URL error: Skipping...********")
        url_errors.append(url_list[i])
        pass

processing Record 1 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=nan&longitude=nan&radius=24140&categories=beaches&sort_by=best_match
********URL error: Skipping...********
processing Record 2 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=nan&longitude=nan&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 3 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=nan&longitude=nan&radius=24140&categories=horsebackriding&sort_by=best_match
********URL error: Skipping...********
processing Record 4 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=nan&longitude=nan&radius=24140&categories=hiking&sort_by=best_match
********URL error: Skipping...********
processing Record 5 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=nan&longitude=nan&radius=24140&categories=hot_air_baloons&sort_by=best_match
********URL error: Skipping...********
processing Record 6 of Set 1 | 

processing Record 2 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=49.8879177&longitude=-119.495902&radius=24140&categories=hot_air_baloons&sort_by=best_match
processing Record 3 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=49.8879177&longitude=-119.495902&radius=24140&categories=paddleboarding&sort_by=best_match
processing Record 4 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=49.8879177&longitude=-119.495902&radius=24140&categories=parasailing&sort_by=best_match
processing Record 5 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=49.8879177&longitude=-119.495902&radius=24140&categories=sailing&sort_by=best_match
processing Record 6 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=49.8879177&longitude=-119.495902&radius=24140&categories=snorkeling&sort_by=best_match
processing Record 7 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=49.8879177&longitude=-119.495902&radius=24140&categories=ziplining

processing Record 49 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=30.2711224&longitude=-87.68938259999999&radius=24140&categories=ziplining&sort_by=best_match
processing Record 0 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=30.2711224&longitude=-87.68938259999999&radius=24140&categories=spas&sort_by=best_match
processing Record 1 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=30.2711224&longitude=-87.68938259999999&radius=24140&categories=hotsprings&sort_by=best_match
processing Record 2 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=30.2711224&longitude=-87.68938259999999&radius=24140&categories=massage&sort_by=best_match
processing Record 3 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=30.2711224&longitude=-87.68938259999999&radius=24140&categories=bedbreakfast&sort_by=best_match
processing Record 4 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=30.2711224&longitude=-87.68938259999999&radiu

processing Record 44 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=15.2214956&longitude=145.73848569999998&radius=24140&categories=massage&sort_by=best_match
processing Record 45 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=15.2214956&longitude=145.73848569999998&radius=24140&categories=bedbreakfast&sort_by=best_match
processing Record 46 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=15.2214956&longitude=145.73848569999998&radius=24140&categories=hotels&sort_by=best_match
processing Record 47 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=15.2214956&longitude=145.73848569999998&radius=24140&categories=resorts&sort_by=best_match
processing Record 48 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=15.2214956&longitude=145.73848569999998&radius=24140&categories=skiresorts&sort_by=best_match
processing Record 49 of Set 3 | https://api.yelp.com/v3/businesses/search?latitude=15.2214956&longitude=145.73848569999998&

processing Record 40 of Set 4 | https://api.yelp.com/v3/businesses/search?latitude=-33.953177600000004&longitude=115.07693689999999&radius=24140&categories=skiresorts&sort_by=best_match
processing Record 41 of Set 4 | https://api.yelp.com/v3/businesses/search?latitude=-33.953177600000004&longitude=115.07693689999999&radius=24140&categories=tours&sort_by=best_match
processing Record 42 of Set 4 | https://api.yelp.com/v3/businesses/search?latitude=-33.953177600000004&longitude=115.07693689999999&radius=24140&categories=transportation&sort_by=best_match
processing Record 43 of Set 4 | https://api.yelp.com/v3/businesses/search?latitude=-33.953177600000004&longitude=115.07693689999999&radius=24140&categories=restaurants&sort_by=best_match
processing Record 44 of Set 4 | https://api.yelp.com/v3/businesses/search?latitude=-33.953177600000004&longitude=115.07693689999999&radius=24140&categories=nightlife&sort_by=best_match
processing Record 45 of Set 4 | https://api.yelp.com/v3/businesses/sear

...

********URL error: Skipping...********
processing Record 47 of Set 81 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=ziplining&sort_by=best_match
********URL error: Skipping...********
processing Record 48 of Set 81 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=spas&sort_by=best_match
********URL error: Skipping...********
processing Record 49 of Set 81 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=hotsprings&sort_by=best_match
********URL error: Skipping...********
processing Record 0 of Set 82 | https://api.yelp.com/v3/businesses/search?latitude=55.860618200000005&longitude=-4.249793299999999&radius=24140&categories=massage&sort_by=best_match
********URL error: Skipping...********
processing Record 1 of Set 82 | https://api.yelp.com/v3/businesses/searc

********URL error: Skipping...********
processing Record 35 of Set 82 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=paddleboarding&sort_by=best_match
********URL error: Skipping...********
processing Record 36 of Set 82 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=parasailing&sort_by=best_match
********URL error: Skipping...********
processing Record 37 of Set 82 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=sailing&sort_by=best_match
********URL error: Skipping...********
processing Record 38 of Set 82 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=24140&categories=snorkeling&sort_by=best_match
********URL error: Skipping...********
processing Record 39 of Set 82 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=

********URL error: Skipping...********
processing Record 23 of Set 83 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 24 of Set 83 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=horsebackriding&sort_by=best_match
********URL error: Skipping...********
processing Record 25 of Set 83 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=hiking&sort_by=best_match
********URL error: Skipping...********
processing Record 26 of Set 83 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=24140&categories=hot_air_baloons&sort_by=best_match
********URL error: Skipping...********
processing Record 27 of Set 83 | https://api.yelp.com/v3/businesses/search?latitude=33

********URL error: Skipping...********
processing Record 12 of Set 84 | https://api.yelp.com/v3/businesses/search?latitude=47.3744489&longitude=8.5410422&radius=24140&categories=restaurants&sort_by=best_match
********URL error: Skipping...********
processing Record 13 of Set 84 | https://api.yelp.com/v3/businesses/search?latitude=47.3744489&longitude=8.5410422&radius=24140&categories=nightlife&sort_by=best_match
********URL error: Skipping...********
processing Record 14 of Set 84 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.146601400000001&radius=24140&categories=beaches&sort_by=best_match
********URL error: Skipping...********
processing Record 15 of Set 84 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.146601400000001&radius=24140&categories=hanggliding&sort_by=best_match
********URL error: Skipping...********
processing Record 16 of Set 84 | https://api.yelp.com/v3/businesses/search?latitude=46.2017559&longitude=6.14660140

********URL error: Skipping...********
processing Record 1 of Set 85 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.513675899999999&radius=24140&categories=skiresorts&sort_by=best_match
********URL error: Skipping...********
processing Record 2 of Set 85 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.513675899999999&radius=24140&categories=tours&sort_by=best_match
********URL error: Skipping...********
processing Record 3 of Set 85 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.513675899999999&radius=24140&categories=transportation&sort_by=best_match
********URL error: Skipping...********
processing Record 4 of Set 85 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.513675899999999&radius=24140&categories=restaurants&sort_by=best_match
********URL error: Skipping...********
processing Record 5 of Set 85 | https://api.yelp.com/v3/businesse

In [14]:
# convert the list of dictionaries to a dataframe
yelp_df = pd.DataFrame(data_list)   
yelp_df

,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,38.046407,-84.497039,beaches,1,1.500000,0.666667
1,38.046407,-84.497039,hanggliding,0,0.000000,0.000000
2,38.046407,-84.497039,horsebackriding,8,3.000000,0.666667
3,38.046407,-84.497039,hiking,4,6.166667,18.666667
4,38.046407,-84.497039,hot_air_baloons,20,28.333333,1723.000000
...,...,...,...,...,...,...
3864,14.594891,120.978262,horsebackriding,0,0.000000,0.000000
3865,14.594891,120.978262,hiking,8,3.333333,1.000000
3866,14.594891,120.978262,hot_air_baloons,20,29.333333,196.666667
3867,14.594891,120.978262,paddleboarding,0,0.000000,0.000000


In [15]:
# determine if there are any missing reviews. At this point, there are missing reviews for four categories
yelp_df["category"].value_counts()

beaches            185
hot_air_baloons    185
hanggliding        185
paddleboarding     185
parasailing        185
hiking             185
massage            184
sailing            184
bedbreakfast       184
nightlife          184
tours              184
skiresorts         184
hotels             184
snorkeling         184
horsebackriding    184
spas               184
hotsprings         184
restaurants        184
transportation     184
resorts            184
ziplining          183
Name: category, dtype: int64

In [16]:
# determine if all datapoints for each 
yelp_df[["latitude","longitude"]].groupby(["latitude","longitude"]).size()

latitude    longitude 
-45.032192  168.661000    21
-37.814218  144.963161    21
-36.852095  174.763180    21
-36.718805  142.220951    21
-35.141813  150.391646    21
                          ..
 56.949398  24.105185     21
 59.325117  18.071093     21
 59.437216  24.745369     21
 59.938732  30.316229     21
 60.394306  5.325919      21
Length: 184, dtype: int64

In [17]:
yelp_df["latitude"].nunique()

184

In [18]:
yelp_df.to_csv("../Resources/yelpdata.csv")